# Ingest STAC Into PgSTAC

This notebook is an example of using `pypgstac` tools to load STAC Collections and Items produced from the other example notebooks into PgSTAC. Once in PgSTAC these data will be available through the STAC API.

This notebook assumes you have defined some environment variables for the PgSTAC database connection. If they are not defined this notebook uses defaults from the [eoAPI](https://github.com/developmentseed/eoapi) Docker Compose setup.

The environment variables expected are the ["standard PG environment variables"](https://github.com/stac-utils/pgstac/blob/v0.9.6/docs/src/pypgstac.md?plain=1#L42-L48) used by PgSTAC,

* `PGHOST`
* `PGPORT`
* `PGUSER`
* `PGDATABASE`
* `PGPASSWORD`

In [1]:
import json
import os
from pathlib import Path
from typing import Iterator

from pypgstac.db import PgstacDB
from pypgstac.load import Loader, Methods

In [2]:
dsn = "postgresql://{username}:{password}@{host}:{port}/{database}".format(
    username=os.getenv("PGUSER", "username"),
    password=os.getenv("PGPASSWORD", "password"),
    host=os.getenv("PGHOST", "localhost"),
    port=os.getenv("PGPORT", 5439),
    database=os.getenv("PGDATABASE", "postgis"),
)

In [3]:
pgstac = PgstacDB(dsn)
loader = Loader(pgstac)

In [4]:
def read_items_from_ndjson(
    ndjson: Path,
    collection_id: str,
) -> Iterator[dict]:
    """Read STAC Items from a NDJSON file for a particular Collection."""
    with ndjson.open() as src:
        for line in src:
            item = json.loads(line)
            item["collection"] = collection_id
            yield item

## OpenAerialMap Collection

In [5]:
loader.load_collections("openaerialmap-collection.json", insert_mode=Methods.upsert)

In [6]:
oam_ndjson = sorted(Path(".").glob("openaerialmap-*.ndjson"))[-1]
print(f"Loading Items from {oam_ndjson}")
loader.load_items(
    read_items_from_ndjson(oam_ndjson, "openaerialmap"),
    insert_mode=Methods.insert_ignore,
)

Loading Items from openaerialmap-20250501T142120.ndjson


## Maxar for OAM Collection

In [7]:
loader.load_collections("maxar-opendata-collection.json", insert_mode=Methods.upsert)

In [8]:
maxar_ndjson = sorted(Path(".").glob("maxar-*.ndjson"))[-1]
print(f"Loading Items from {maxar_ndjson}")
loader.load_items(
    read_items_from_ndjson(maxar_ndjson, "maxar-opendata"),
    insert_mode=Methods.insert_ignore,
)

Loading Items from maxar-opendata-20250501T141434.ndjson
